In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# @title RBF based Neural Network for Classification
iris = load_iris()
X_cls = iris.data
y_cls = iris.target

In [3]:
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(X_cls, y_cls, test_size=0.2, random_state=42)

In [4]:
class RBFNClassifier:
    def __init__(self, num_centers, gamma=1.0, random_state=None):
        self.num_centers = num_centers
        self.gamma = gamma
        self.random_state = random_state
        self.centers = None
        self.weights = None
        self.encoder = OneHotEncoder()

    def _gaussian(self, X, centers):
        return np.exp(-self.gamma * euclidean_distances(X, centers) ** 2)

    def fit(self, X, y):
        rng = np.random.RandomState(self.random_state)
        self.centers = rng.permutation(X)[:self.num_centers]
        X_transformed = self._gaussian(X, self.centers)
        self.encoder.fit(y.reshape(-1, 1))
        y_encoded = self.encoder.transform(y.reshape(-1, 1)).toarray()
        self.weights = np.linalg.pinv(X_transformed) @ y_encoded

    def predict(self, X):
        X_transformed = self._gaussian(X, self.centers)
        preds = X_transformed @ self.weights
        return self.encoder.inverse_transform(preds)

In [5]:
def evaluate_classification_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    return accuracy, precision, recall, f1

In [6]:
# Radial Basis Function
rbfn_classifier = RBFNClassifier(num_centers=10, gamma=0.1, random_state=42)
rbfn_classifier.fit(X_train_cls, y_train_cls)
rbfn_preds = rbfn_classifier.predict(X_test_cls)
rbfn_metrics = evaluate_classification_model(y_test_cls, rbfn_preds)

In [7]:
# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train_cls, y_train_cls)
log_reg_preds = log_reg.predict(X_test_cls)
log_reg_metrics = evaluate_classification_model(y_test_cls, log_reg_preds)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
# Decision Trees
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train_cls, y_train_cls)
dt_preds = dt_classifier.predict(X_test_cls)
dt_metrics = evaluate_classification_model(y_test_cls, dt_preds)

In [9]:
# Support Vector Machines (SVMs)
svm_classifier = SVC()
svm_classifier.fit(X_train_cls, y_train_cls)
svm_preds = svm_classifier.predict(X_test_cls)
svm_metrics = evaluate_classification_model(y_test_cls, svm_preds)

In [10]:
# Print the accuracies of each model
print("Logistic Regression Accuracy:", log_reg_metrics)
print("Decision Trees Accuracy:", dt_metrics)
print("SVM Accuracy:", svm_metrics)
print("RBFN Accuracy:", rbfn_metrics)

Logistic Regression Accuracy: (1.0, 1.0, 1.0, 1.0)
Decision Trees Accuracy: (1.0, 1.0, 1.0, 1.0)
SVM Accuracy: (1.0, 1.0, 1.0, 1.0)
RBFN Accuracy: (1.0, 1.0, 1.0, 1.0)


In [11]:
# @title RBF based Neural Network for Regression
# Load the Boston housing dataset from the original source
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

In [12]:
# Extract features and target from the raw data
X_reg = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y_reg = raw_df.values[1::2, 2]

In [13]:
print(X_reg.shape, y_reg.shape)

(506, 13) (506,)


In [14]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

In [15]:
class RBFNRegressor:
    def __init__(self, num_centers, gamma=1.0, random_state=None):
        self.num_centers = num_centers
        self.gamma = gamma
        self.random_state = random_state
        self.centers = None
        self.weights = None

    def _gaussian(self, X, centers):
        return np.exp(-self.gamma * euclidean_distances(X, centers) ** 2)

    def fit(self, X, y):
        rng = np.random.RandomState(self.random_state)
        self.centers = rng.permutation(X)[:self.num_centers]
        X_transformed = self._gaussian(X, self.centers)
        self.weights = np.linalg.pinv(X_transformed) @ y

    def predict(self, X):
        X_transformed = self._gaussian(X, self.centers)
        return X_transformed @ self.weights

def evaluate_regression_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, r2

In [16]:
# Radial Basis Function
rbfn_regressor = RBFNRegressor(num_centers=10, gamma=0.1, random_state=42)
rbfn_regressor.fit(X_train_reg, y_train_reg)
rbfn_preds = rbfn_regressor.predict(X_test_reg)
rbfn_metrics = evaluate_regression_model(y_test_reg, rbfn_preds)

In [17]:
# Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(X_train_reg, y_train_reg)
lin_reg_preds = lin_reg.predict(X_test_reg)
lin_reg_metrics = evaluate_regression_model(y_test_reg, lin_reg_preds)

In [18]:
# Decision Tree
dt_regressor = DecisionTreeRegressor()
dt_regressor.fit(X_train_reg, y_train_reg)
dt_preds = dt_regressor.predict(X_test_reg)
dt_metrics = evaluate_regression_model(y_test_reg, dt_preds)

In [19]:
# Random Forests
rf_regressor = RandomForestRegressor()
rf_regressor.fit(X_train_reg, y_train_reg)
rf_preds = rf_regressor.predict(X_test_reg)
rf_metrics = evaluate_regression_model(y_test_reg, rf_preds)

In [20]:
print("Linear Regression Metrics:", lin_reg_metrics)
print("Random Forest Metrics:", rf_metrics)
print("Decision Tree Metrics:", dt_metrics)
print("RBFN Metrics:", rbfn_metrics)

Linear Regression Metrics: (24.291119474973485, 4.928602182665333, 0.6687594935356325)
Random Forest Metrics: (8.553326441176472, 2.924607057567986, 0.8833644457905967)
Decision Tree Metrics: (9.721764705882354, 3.117974455617357, 0.8674312944604461)
RBFN Metrics: (528.723345434056, 22.99398498377469, -6.209819576308458)
